In [24]:
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from tinymlgen import port

In [25]:
all_csvs = []

for i in range(0, 12):
  position_csv = pd.read_csv("../gyro_readings/position_" + str(i) + ".csv")
  position_csv["value"] = i
  all_csvs.append(position_csv)

combined_csv = pd.concat(all_csvs)
combined_csv.head()

shuffled_csv = combined_csv.sample(frac=1)
training_csv = shuffled_csv[:-100]
eval_csv = shuffled_csv[-100:]

print(training_csv)

     accel_x   accel_y   accel_z    gyro_x    gyro_y    gyro_z  value
37  4.375276  3.388835 -7.695916  0.048106 -0.139583 -0.085521      5
11 -4.235895 -3.658028  8.423933  0.050244 -0.140041 -0.083841      8
33 -4.440480  7.354939 -4.102495  0.049480 -0.138056 -0.085979      6
8  -3.396612 -1.869991 -7.149755  0.722348  0.234267  1.628872     10
19 -4.295715 -2.039881 -8.191230 -0.028253 -0.155771 -0.040470     10
..       ...       ...       ...       ...       ...       ...    ...
24 -4.280162 -3.538387  8.453843  0.050244 -0.140652 -0.083841      8
15 -4.348955 -3.597609  8.369495  0.055589 -0.134238 -0.079871      8
27 -4.539185  6.452246  5.997012  0.046578 -0.137445 -0.082314      7
22  9.754342 -0.415155  0.629312  0.046731 -0.138056 -0.087506      0
86  4.366901  3.516851 -7.652845  0.051465 -0.136070 -0.084147      5

[370 rows x 7 columns]


In [26]:
positions_features = training_csv.copy()
positions_labels = positions_features.pop('value')

eval_positions_features = eval_csv.copy()
eval_positions_labels = eval_positions_features.pop('value')

In [27]:
positions_features = np.array(positions_features)
positions_features

array([[ 4.375,  3.389, -7.696,  0.048, -0.14 , -0.086],
       [-4.236, -3.658,  8.424,  0.05 , -0.14 , -0.084],
       [-4.44 ,  7.355, -4.102,  0.049, -0.138, -0.086],
       ...,
       [-4.539,  6.452,  5.997,  0.047, -0.137, -0.082],
       [ 9.754, -0.415,  0.629,  0.047, -0.138, -0.088],
       [ 4.367,  3.517, -7.653,  0.051, -0.136, -0.084]])

In [28]:
normalize = preprocessing.Normalization()
normalize.adapt(positions_features)

In [29]:
positions_model = tf.keras.Sequential([
  layers.Dense(6),
  layers.Dense(12, activation='softmax')
])

positions_model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                      optimizer = tf.keras.optimizers.Adam())

In [30]:
positions_model.fit(positions_features, positions_labels, epochs=100, validation_data=(eval_positions_features, eval_positions_labels))

Epoch 1/100
 1/12 [=>............................] - ETA: 0s - loss: 4.3191WARNING:tensorflow:Layer dense_6 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

12/12 [==============================] - 0s 9ms/step - loss: 4.1926 - val_loss: 3.8392
Epoch 2/100
12/12 [==============================] - 0s 4ms/step - loss: 3.8521 - val_loss: 3.5438
Epoch 3/100
12/12 [=========

In [31]:
positions_model.predict([[-4.067799,-8.872587,-0.638285,0.036805,-0.132405,-0.085521]])

array([[0.   , 0.   , 0.   , 0.004, 0.014, 0.   , 0.   , 0.   , 0.006,
        0.937, 0.025, 0.014]], dtype=float32)

In [32]:
# tf.keras.saved_model.save(positions_model, "./model/")

AttributeError: module 'tensorflow.keras' has no attribute 'saved_model'

In [748]:
    c_code = port(positions_model, pretty_print=True, optimize=True)
    print(c_code)

INFO:tensorflow:Assets written to: /var/folders/1q/_vlxftdn3gn_h6kg3ty1b2jr0000gn/T/tmpkccq4jw2/assets
INFO:tensorflow:Assets written to: /var/folders/1q/_vlxftdn3gn_h6kg3ty1b2jr0000gn/T/tmpkccq4jw2/assets

#ifdef __has_attribute
#define HAVE_ATTRIBUTE(x) __has_attribute(x)
#else
#define HAVE_ATTRIBUTE(x) 0
#endif
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__))
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4)))
#else
#define DATA_ALIGN_ATTRIBUTE
#endif

const unsigned char model_data[] DATA_ALIGN_ATTRIBUTE = {
	0x28, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x00, 0x00, 
	0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x12, 0x00, 
	0x1c, 0x00, 0x04, 0x00, 0x08, 0x00, 0x0c, 0x00, 0x10, 0x00, 0x14, 0x00, 
	0x00, 0x00, 0x18, 0x00, 0x12, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00, 
	0x50, 0x07, 0x00, 0x00, 0x28, 0x03, 0x00, 0x00, 0x10, 0x03, 0x00, 0x00, 
	0x3c, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 
	0x0c, 0

In [749]:
converter = tf.lite.TFLiteConverter.from_keras_model(positions_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with tf.io.gfile.GFile('test_model.tflite', 'wb') as f:
      f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/1q/_vlxftdn3gn_h6kg3ty1b2jr0000gn/T/tmpx8vps9is/assets
INFO:tensorflow:Assets written to: /var/folders/1q/_vlxftdn3gn_h6kg3ty1b2jr0000gn/T/tmpx8vps9is/assets
